In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# get excel file in a xls variable
xls = pd.ExcelFile("../Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx")


In [2]:
vaccinationer_tidsserie = pd.read_excel(xls, 'Vaccinationer tidsserie')
vaccinerade_tidsserie = pd.read_excel(xls, 'Vaccinerade tidsserie')
vaccinerade_dos_3 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 3')
vaccinerade_dos_4 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 4')
vaccinerade_dos_5 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 5')
dos_1_to_3_per_age_group = pd.read_excel(xls, 'Dos 1 till 3 per åldersgrupp')
dos_4_per_age_group = pd.read_excel(xls, 'Dos 4 per åldersgrupp')
dos_4_18_plus = pd.read_excel(xls, 'Dos 4 18+')
dos_5_per_age_group  = pd.read_excel(xls, 'Dos 5 per åldersgrupp')
vaccinerade_gender  = pd.read_excel(xls, 'Vaccinerade kön')
vaccinerade_kommun  = pd.read_excel(xls, 'Vaccinerade kommun')
vaccinerade_kommun_dos_3  = pd.read_excel(xls, 'Vaccinerade kommun dos 3')
vaccinerade_kommun_dos_4  = pd.read_excel(xls, 'Vaccinerade kommun dos 4')
vaccinerade_kommun_dos_5  = pd.read_excel(xls, 'Vaccinerade kommun dos 5')
vaccinerade_kommun_och_ålder  = pd.read_excel(xls, 'Vaccinerade kommun och ålder')



In [3]:
vaccinationer_tidsserie.head()

,Vecka,År,Region,Antal vaccinationer
0,52,2020,| Sverige |,2478
1,52,2020,Stockholm,289
2,52,2020,Uppsala,9
3,52,2020,Södermanland,116
4,52,2020,Östergötland,214


In [4]:
vaccinerade_tidsserie['Region'].value_counts

<bound method IndexOpsMixin.value_counts of 0        | Sverige |
1        | Sverige |
2          Stockholm
3          Stockholm
4            Uppsala
            ...     
4219        Jämtland
4220    Västerbotten
4221    Västerbotten
4222      Norrbotten
4223      Norrbotten
Name: Region, Length: 4224, dtype: object>

In [5]:
vaccinerade_kommun_och_ålder.head(3)

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN


## 2.a

In [6]:
antal_län = vaccinerade_kommun_och_ålder['Län'].nunique()
print(f"Antal län i sverige = {antal_län}")

Antal län i sverige = 21


## 2.b

In [7]:
antal_Kommuner = vaccinerade_kommun_och_ålder['Kommun'].nunique()
print(f"Antal Kommuner i sverige = {antal_Kommuner}")

Antal Kommuner i sverige = 290


In [8]:
total_population = 10_502_959
print(f"Total befolkning i sverige = {total_population}")

Total befolkning i sverige = 10502959


In [9]:
vaccinerade_kommun_och_ålder.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN


In [10]:
vaccinerade_kommun_och_ålder["Ålder"] =  vaccinerade_kommun_och_ålder["Ålder"].str[:2].astype(int)
vaccinerade_kommun_och_ålder["Ålder"].head()

0    12
1    16
2    18
3    30
4    40
Name: Ålder, dtype: int32

## 2. d)

In [11]:
import numpy as np

#https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas
over_17 = np.where(vaccinerade_kommun_och_ålder["Ålder"] >= 18, vaccinerade_kommun_och_ålder['Befolkning'], 0).sum()

In [12]:
under_18 = total_population - over_17
under_18
f'Number of people under 18 = {under_18:,}'

'Number of people under 18 = 2,155,539'

## 2. e)

In [13]:
dos_1 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel minst 1 dos'].sum()
dos_1.head(3)

Län_namn
Blekinge län     42.822781
Dalarnas län    128.333151
Gotlands län      8.903401
Name: Andel minst 1 dos, dtype: float64

### minst dos 1

In [14]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos1 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel minst 1 dos'].mean() 
fig = px.bar(x=minst_dos1.index, y=minst_dos1, title="Mean Andel med minst 1 dos")
fig.update_yaxes(title="Andelen")
fig.update_xaxes(title="Län")
fig.show()



### minst dos 2

In [15]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos2 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel minst 2 doser'].mean()

fig = px.bar(x=minst_dos2.index, y=minst_dos2, title="Andel minst 2 doser")
fig.update_yaxes(title="Andelen")
fig.update_xaxes(title="Län")
fig.show()

### minst 3 doser

In [16]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos3 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel 3 doser'].mean()

fig = px.bar(x=minst_dos3.index, y=minst_dos3)
fig.update_yaxes(title="Andelen",showgrid=False,showticklabels=True)
fig.update_xaxes(title="Län", showgrid=False, zeroline=True,showticklabels=True, showline=False)
fig.show()



In [17]:
vaccinerade_kommun_och_ålder.tail(1)

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
2899,25,Norrbottens län,2584,Kiruna,90,219,206,206,200.0,172.0,0.940639,0.940639,0.913242,0.785388


In [18]:
stockholm_län =vaccinerade_kommun_och_ålder[vaccinerade_kommun_och_ålder["Län_namn"] == "Stockholms län"]
stockholm_län.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN


In [19]:
västra_götaland_län =vaccinerade_kommun_och_ålder[vaccinerade_kommun_och_ålder["Län_namn"] == "Västra Götalands län"]
västra_götaland_län.head()


,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
1340,14,Västra Götalands län,1401,Härryda,12,2641,1615,1508,NaN,NaN,0.611511,0.570996,NaN,NaN
1341,14,Västra Götalands län,1401,Härryda,16,1123,988,945,NaN,NaN,0.879786,0.841496,NaN,NaN
1342,14,Västra Götalands län,1401,Härryda,18,4446,3645,3480,1617.0,NaN,0.819838,0.782726,0.363698,NaN
1343,14,Västra Götalands län,1401,Härryda,30,4844,4106,3985,2299.0,NaN,0.847647,0.822667,0.474608,NaN
1344,14,Västra Götalands län,1401,Härryda,40,6078,5502,5439,4007.0,NaN,0.905232,0.894867,0.659263,NaN


In [20]:
västra_götaland_län['Andel 3 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_848\4057759334.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
västra_götaland_län['Andel 4 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_848\4203142793.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
stockholm_län['Andel 3 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_848\71766071.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
stockholm_län['Andel 4 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_848\1147220974.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
the_big_2 = pd.concat([västra_götaland_län, stockholm_län])
the_big_2.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
1340,14,Västra Götalands län,1401,Härryda,12,2641,1615,1508,NaN,NaN,0.611511,0.570996,0.000000,0.0
1341,14,Västra Götalands län,1401,Härryda,16,1123,988,945,NaN,NaN,0.879786,0.841496,0.000000,0.0
1342,14,Västra Götalands län,1401,Härryda,18,4446,3645,3480,1617.0,NaN,0.819838,0.782726,0.363698,0.0
1343,14,Västra Götalands län,1401,Härryda,30,4844,4106,3985,2299.0,NaN,0.847647,0.822667,0.474608,0.0
1344,14,Västra Götalands län,1401,Härryda,40,6078,5502,5439,4007.0,NaN,0.905232,0.894867,0.659263,0.0


In [25]:
df = the_big_2.groupby('Län_namn').agg({'Andel minst 1 dos':'mean', 'Andel minst 2 doser':'mean', 'Andel 3 doser':'mean', 'Andel 4 doser':'mean'}).T

In [26]:
df.index

Index(['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser',
       'Andel 4 doser'],
      dtype='object')

In [31]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot(df: pd.DataFrame):
    columns = ['Stockholms län', 'Västra Götalands län']
    fig = make_subplots(rows=1, cols=1, subplot_titles=["Mean andel dos per region"])
    for i, col in enumerate(columns):
        fig.add_trace(
            go.Bar(x=df.index, y=df[col],
            name=col, showlegend=True)
            
        )
    fig.update_layout(showlegend=True)
    fig.update_yaxes(tickformat=",.0%", range=[0,1], title="mean andel dos")
    fig.update_xaxes(title="mean andel dos per län")
    fig.show()

In [32]:
plot(df)